First, import needed modules

In [193]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [194]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [195]:
btw94_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW94_Erststimmen_Gemeinden.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw98_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW98_Erststimmen_Gemeinden.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

btw94_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW94_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw98_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW98_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw02_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW02_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW05_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW09_Zweitstimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW13_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_zs = pd.read_csv(f'{path}/data/elections/btw/zs/btw17_wbz_zweitstimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_95507/115399916.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_95507/115399916.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_95507/115399916.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_csv(

Create and convert all AGS to the AGS in 2013, add year column 

In [196]:
ags_trans = pd.read_csv(f'{path}/data/ags_transition.csv')

In [197]:
dfs = [btw94_es, btw98_es, btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw94_zs, btw98_zs, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
years = [1994, 1998, 2002, 2005, 2009, 2013, 2017, 1994, 1998, 2002, 2005, 2009, 2013, 2017]
for i, (year, df) in enumerate(zip(years, dfs)):
    # insert year column
    dfs[i].insert(loc=0, column='year', value=year)
    # create AGS: it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns
    dfs[i].insert(loc=0, column='AGS', value=0)
    dfs[i]['AGS'] = dfs[i]['Land'] + dfs[i]['Regierungsbezirk'] + dfs[i]['Kreis'] + dfs[i]['Gemeinde']
    # Sometimes the AGS of previous years are used (likely because electroral districts are decided on quite early)
    # read in AGS transition datasets for same and previous year
    ags_trans_same = pd.read_csv(f'{path}/data/ags_transition.csv', usecols=[f'AGS_{year}', 'AGS_2013'], converters={f'AGS_{year}': str, 'AGS_2013': str})
    ags_trans_same.rename(columns = {f'AGS_{year}':'AGS_new'}, inplace = True)
    ags_trans_prev = pd.read_csv(f'{path}/data/ags_transition.csv', usecols=[f'AGS_{year-1}', 'AGS_2013'], converters={f'AGS_{year-1}': str, 'AGS_2013': str})
    ags_trans_prev.rename(columns = {f'AGS_{year-1}':'AGS_new'}, inplace = True)
    # concat to one df
    ags_trans = pd.concat([ags_trans_same, ags_trans_prev])
    # merge dfs[i] with ags trans
    dfs[i] = dfs[i].merge(ags_trans, left_on='AGS', right_on='AGS_new', how='left', indicator=True)
    print(dfs[i]._merge.value_counts())
    # if merged successfully, replace AGS
    dfs[i]['AGS'] = np.where(dfs[i]['AGS_2013'].notna(), dfs[i]['AGS_2013'], dfs[i]['AGS'])
    dfs[i] = dfs[i].drop(['AGS_2013', 'AGS_new', '_merge'], axis=1)
    # move AGS to front
    AGS = dfs[i].pop('AGS')
    dfs[i].insert(1, 'AGS', AGS)
    # the merge produces duplicates --> drop
    dfs[i] = dfs[i].drop_duplicates(keep='first')
# unpack list back to variables
btw94_es, btw98_es, btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw94_zs, btw98_zs, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs = dfs
#btw_ags_trans = pd.concat(dfs)

both          28815
left_only       542
right_only        0
Name: _merge, dtype: int64
both          32944
left_only       405
right_only        0
Name: _merge, dtype: int64
both          282569
left_only       4224
right_only         0
Name: _merge, dtype: int64
both          310506
left_only       4160
right_only         0
Name: _merge, dtype: int64
both          322634
left_only       3488
right_only         0
Name: _merge, dtype: int64
both          358616
left_only       3185
right_only         0
Name: _merge, dtype: int64
both          365560
left_only       3415
right_only         0
Name: _merge, dtype: int64
both          185948
left_only       3853
right_only         0
Name: _merge, dtype: int64
both          233340
left_only       3939
right_only         0
Name: _merge, dtype: int64
both          282569
left_only       4224
right_only         0
Name: _merge, dtype: int64
both          310506
left_only       4160
right_only         0
Name: _merge, dtype: int64
both          32

Since some municipalities share a ballot vote election board, in this case, postal ballots cannot be attributed to a single municipality. 
In the 1994 to 2017 data, these cases were signified with '999' as Gemeindekennziffer. From 2017 on the Gemeindekennziffer is given by 9 + Kennziffer Briefwahlzugehörigkeit.

To drop the 999 Gemeinde in for 1994-2013, first append these dataframes

In [198]:
dfs = [btw94_es, btw98_es, btw02_es, btw05_es, btw09_es, btw13_es]
btw9413_es = pd.concat(dfs, sort=False)
dfs = [btw94_zs, btw98_zs, btw02_zs, btw05_zs, btw09_zs, btw13_zs]
btw9413_zs = pd.concat(dfs, sort=False)
btw9413_zs.shape

(535761, 88)

Then drop all observations with Gemeinde == 999

In [199]:
btw9413_es = btw9413_es[btw9413_es['Gemeinde'] != '999']
btw9413_zs = btw9413_zs[btw9413_zs['Gemeinde'] != '999']
btw9413_zs.shape

(529717, 88)

For 2017, drop all observations where Gemeinde == 9 + Kennziffer Briefwahlzugehörigkeit

In [200]:
btw17_es = btw17_es[btw17_es['Gemeinde'] != '9' + btw17_es['Kennziffer Briefwahlzugehörigkeit']]
btw17_zs = btw17_zs[btw17_zs['Gemeinde'] != '9' + btw17_zs['Kennziffer Briefwahlzugehörigkeit']]

Now contatenate the dfs for 1994-2013 with 2017

In [201]:
dfs = [btw9413_es, btw17_es]
btw_es_raw = pd.concat(dfs, sort=False)
dfs = [btw9413_zs, btw17_zs]
btw_zs_raw = pd.concat(dfs, sort=False)

Since there is only one polygon with the AGS 11000000 for Berlin, replace AGS values here

In [202]:
btw_es_raw['AGS'] = np.where(btw_es_raw['Land'] == '11', '11000000', btw_es_raw['AGS']) #if condition, = 110..., else AGS
btw_zs_raw['AGS'] = np.where(btw_zs_raw['Land'] == '11', '11000000', btw_zs_raw['AGS'])

Since Kreis Numbers repeat within states, concatenate State Number and Kreis Number to make them unique

In [203]:
len(btw_es_raw['Kreis'].unique())

92

In [204]:
btw_es_raw['Kreis'] = btw_es_raw['Land'] + btw_es_raw['Kreis']
btw_zs_raw['Kreis'] = btw_zs_raw['Land'] + btw_zs_raw['Kreis']
len(btw_es_raw['Kreis'].unique()) # In 2021, there are 294 Kreise in DE

301

In [205]:
len(btw_es_raw['Wahlkreis'].unique())

328

Collapse data along AGS column, sums votes over all constituencies in a municipality

In [206]:
es_cols = list(btw_es_raw.columns)
es_cols_to_keep = list(btw_es_raw.select_dtypes(['object']).columns) + ['Wahlkreis', 'Bezirksart'] # keep these columns
es_cols_to_sum = [i for i in es_cols if i not in es_cols_to_keep] # sum these columns
es_cols_to_sum.remove('year')
zs_cols = list(btw_zs_raw.columns)
zs_cols_to_keep = list(btw_zs_raw.select_dtypes(['object']).columns) + ['Wahlkreis', 'Bezirksart']
zs_cols_to_sum = [i for i in zs_cols if i not in zs_cols_to_keep]
zs_cols_to_sum.remove('year')

In [207]:
btw_es_coll = btw_es_raw.groupby(['AGS', 'year'], as_index=False).agg({**{e:'first' for e in es_cols_to_keep}, **{e:'sum' for e in es_cols_to_sum}})
btw_zs_coll = btw_zs_raw.groupby(['AGS', 'year'], as_index=False).agg({**{e:'first' for e in zs_cols_to_keep}, **{e:'sum' for e in zs_cols_to_sum}})
btw_es_coll.head()

,year,AGS,Land,Regierungsbezirk,Kreis,Gemeinde,Wahlbezirk,Kennziffer Briefwahlzugehörigkeit,Ungekürzte Wahlbezirksbezeichnung,Wahlkreis,...,Die Grauen,du,MG,MENSCHLICHE WELT,Gesundheitsforschung,V-Partei³,Bündnis C,DIE EINHEIT,MIETERPARTEI,Neue Liberale
0,1994,01001000,01,0,0101,000,None,None,None,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1998,01001000,01,0,0101,000,None,None,None,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2002,01001000,01,0,0101,000,1,None,None,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2005,01001000,01,0,0101,000,000001,0.0,None,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,01001000,01,0,0101,000,000025,0.0,None,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Inspect the distribution of obs. over years

In [208]:
btw_es_coll['year'].value_counts()

1994    11167
2009    11163
1998    11161
2005    11160
2002    11159
2013    11159
2017    11157
Name: year, dtype: int64

In [209]:
btw_zs_coll['year'].value_counts()

1994    11167
2009    11163
1998    11161
2005    11160
2002    11159
2013    11159
2017    11157
Name: year, dtype: int64

Add indicator for first and second vote

In [210]:
dfs = [btw_es_coll, btw_zs_coll]
eszs = ['first_vote', 'second_vote']
for df, stimme in zip(dfs, eszs):
    df.insert(loc=2, column=stimme, value=1)
btw_zs_coll.head()

,year,AGS,second_vote,Land,Regierungsbezirk,Kreis,Gemeinde,Wahlbezirk,Kennziffer Briefwahlzugehörigkeit,Ungekürzte Wahlbezirksbezeichnung,...,BGE,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³
0,1994,01001000,1,01,0,0101,000,1,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1998,01001000,1,01,0,0101,000,29,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2002,01001000,1,01,0,0101,000,1,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2005,01001000,1,01,0,0101,000,000001,0.0,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,01001000,1,01,0,0101,000,000001,0.0,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ES: Sum votes for other, Union and Die Linke parties

In [211]:
# First declare the columns to sum
es_col = list(btw_es_coll.columns)
union = ['CDU', 'CSU']
linke = ['Die Linke', 'DIE LINKE', 'PDS']
invalid = ['Ungültige', 'Ungültig']
valid = ['Gültige', 'Gültig']
other_keep = ['year',
 'AGS',
 'first_vote',
 'Land',
 'Regierungsbezirk',
 'Kreis',
 'Gemeinde',
 'Wahlbezirk',
 'Kennziffer Briefwahlzugehörigkeit',
 'Ungekürzte Wahlbezirksbezeichnung',
 'Wahlkreis',
 'Bezirksart',
 'Verbandsgemeinde',
 'Wahlberechtigte (A)',
 'Wahlberechtigte ohne Sperrvermerk (A1)',
 'Wahlberechtigte mit Sperrvermerk (A2)',
 'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)',
 'Wähler (B)',
 'Wähler mit Wahlschein (B1)', 'SPD', 'GRÜNE', 'FDP']
es_cols_to_keep = union + linke + invalid + valid + other_keep
andere = list(set(es_col) ^ set(es_cols_to_keep))

In [212]:
print(andere)

['NPD', 'FORUM', 'HUMANWIRTSCHAFTSPARTEI', 'ÖDP', 'ÜBRIGE', 'Tierschutzallianz', 'ADM', 'Die PARTEI', 'PARTEI DER VERNUNFT', 'DIE PARTEI', 'Freie Union', 'DSU', 'VIOLETTEN', 'UNABHÄNGIGE', 'GRAUE', 'NATURGESETZ', 'FBU', 'ZENTRUM', 'FP D', 'UAP', 'FSU', 'APD', 'AGFG', 'Pro DM', 'HP', 'BÜSO', 'Bündnis 21/RRP', 'CHANCE 2000', 'PBC', 'REP', 'DMP', 'Nichtwähler', 'FRAUEN', 'BfB', 'DPD', 'BP', 'NEIN', 'LIGA', 'MLPD', 'DKP', 'PSG', 'PASS', 'Deutschland', 'PRG', 'Die Grauen', 'AUFBRUCH', 'KPD', 'RENTNER', 'DVP', 'AfD', 'MG', 'pro Deutschland', 'BGD', 'V-Partei³', 'DIE VIOLETTEN', 'Die Tierschutzpartei', 'du', 'Solidarität', 'DVU', 'Volksabstimmung', 'Übrige', 'Gesundheitsforschung', 'Offensive D', 'FP Deutschland', 'Tierschutzpartei', 'MENSCHLICHE WELT', 'DEUTSCHLAND', 'BIG', 'DEMOKRATEN', 'SCHILL', 'AB 2000', 'BüSo', 'DIE EINHEIT', 'CM', 'ödp', 'FAMILIE', 'WGr/EB', 'LD', 'DIE FRAUEN', 'Neue Liberale', 'RRP', 'APPD', 'TIERSCHUTZ', 'DIE RECHTE', 'PDV', 'SGP', 'PIRATEN', 'MIETERPARTEI', 'STATT P

In [213]:
btw_es_sum = btw_es_coll.copy()
btw_es_sum['Andere'] = btw_es_sum[andere].sum(axis=1) # sum votes for 'Other' Parties
btw_es_sum['Union'] = btw_es_sum[union].sum(axis=1) # sum votes for 'Union' Parties
btw_es_sum['Linke'] = btw_es_sum[linke].sum(axis=1) # sum votes for 'Linke' Parties
btw_es_sum['Ungültig'] = btw_es_sum[invalid].sum(axis=1) # sum invalid votes
btw_es_sum['Gültig'] = btw_es_sum[valid].sum(axis=1) # sum valid votes

Drop not needed columns

In [214]:
columns_to_drop = andere + union + linke + ['Ungültige', 'Gültige', 'Verbandsgemeinde', 'Regierungsbezirk', 'Gemeinde', 'Wahlbezirk', 'Kennziffer Briefwahlzugehörigkeit', 'Ungekürzte Wahlbezirksbezeichnung']
btw_es_sum.drop(columns_to_drop, axis=1, inplace=True)
btw_es_sum.columns

Index(['year', 'AGS', 'first_vote', 'Land', 'Kreis', 'Wahlkreis', 'Bezirksart',
       'Wahlberechtigte (A)', 'Wahlberechtigte ohne Sperrvermerk (A1)',
       'Wahlberechtigte mit Sperrvermerk (A2)',
       'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)', 'Wähler (B)',
       'Wähler mit Wahlschein (B1)', 'Ungültig', 'Gültig', 'SPD', 'FDP',
       'GRÜNE', 'Andere', 'Union', 'Linke'],
      dtype='object')

ZS: Sum votes for other, Union and DIE LINKE parties

In [215]:
# First declare the columns to sum
zs_col = list(btw_zs_coll.columns)
union = ['CDU', 'CSU']
linke = ['Die Linke', 'DIE LINKE', 'PDS']
invalid = ['Ungültige', 'Ungültig']
valid = ['Gültige', 'Gültig']
other_keep = ['year',
 'AGS',
 'second_vote',
 'Land',
 'Regierungsbezirk',
 'Kreis',
 'Gemeinde',
 'Wahlbezirk',
 'Kennziffer Briefwahlzugehörigkeit',
 'Ungekürzte Wahlbezirksbezeichnung',
 'Wahlkreis',
 'Bezirksart',
 'Verbandsgemeinde',
 'Wahlberechtigte (A)',
 'Wahlberechtigte ohne Sperrvermerk (A1)',
 'Wahlberechtigte mit Sperrvermerk (A2)',
 'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)',
 'Wähler (B)',
 'Wähler mit Wahlschein (B1)', 'SPD', 'GRÜNE', 'FDP']
zs_cols_to_keep = union + linke + invalid + valid + other_keep
andere = list(set(zs_col) ^ set(zs_cols_to_keep))

In [216]:
print(andere)

['NPD', 'FORUM', 'ÖDP', 'Tierschutzallianz', 'ADM', 'Die PARTEI', 'BGE', 'PARTEI DER VERNUNFT', '50plus', 'Violetten', 'DM', 'DiB', 'GRAUE', 'FWD', 'NATURGESETZ', 'ZENTRUM', 'APD', 'AGFG', 'Pro DM', 'HP', 'Bündnis 21/RRP', 'Partei der Nichtwähler', 'CHANCE 2000', 'PBC', 'REP', 'Nichtwähler', 'FRAUEN', 'BfB', 'DPD', 'BP', 'Die Humanisten', 'LIGA', 'MLPD', 'DKP', 'PSG', 'PASS', 'Deutschland', 'AUFBRUCH', 'Die Grauen', 'PRG', 'KPD', 'RENTNER', 'AfD', 'MG', 'pro Deutschland', 'V-Partei³', 'DIE VIOLETTEN', 'Die Tierschutzpartei', 'DVU', 'Volksabstimmung', 'Schill', 'Gesundheitsforschung', 'Offensive D', 'Tierschutzpartei', 'Tierschutz', 'MENSCHLICHE WELT', 'Die Partei', 'BIG', 'du.', 'AB 2000', 'AD-DEMOKRATEN', 'BüSo', 'CM', 'ödp', 'FAMILIE', 'BSA', 'DIE FRAUEN', 'RRP', 'APPD', 'DIE RECHTE', 'PDV', 'SGP', 'PIRATEN', 'STATT Partei', 'B*', 'FREIE WÄHLER']


In [217]:
btw_zs_sum = btw_zs_coll.copy()
btw_zs_sum['Andere'] = btw_zs_sum[andere].sum(axis=1) # sum votes for 'Other' Parties
btw_zs_sum['Union'] = btw_zs_sum[union].sum(axis=1) # sum votes for 'Union' Parties
btw_zs_sum['Linke'] = btw_zs_sum[linke].sum(axis=1) # sum votes for 'Linke' Parties
btw_zs_sum['Ungültig'] = btw_zs_sum[invalid].sum(axis=1) # sum invalid votes
btw_zs_sum['Gültig'] = btw_zs_sum[valid].sum(axis=1) # sum valid votes

In [218]:
columns_to_drop = andere + union + linke + ['Ungültige', 'Gültige', 'Verbandsgemeinde', 'Regierungsbezirk', 'Gemeinde', 'Wahlbezirk', 'Kennziffer Briefwahlzugehörigkeit', 'Ungekürzte Wahlbezirksbezeichnung']
btw_zs_sum.drop(columns_to_drop, axis=1, inplace=True)
btw_zs_sum.head()

,year,AGS,second_vote,Land,Kreis,Wahlkreis,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,Wähler (B),Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,FDP,GRÜNE,Andere,Union,Linke
0,1994,01001000,1,01,0101,1,0,68987,61031,7956,...,52379,7535,556.0,51823.0,24172,3404,4651,1059.0,17837,700.0
1,1998,01001000,1,01,0101,1,0,65755,58545,7210,...,50560,6678,698.0,49862.0,25490,3182,3815,1925.0,14538,912.0
2,2002,01001000,1,01,0101,1,0,65740,57788,7949,...,49054,7573,501.0,48553.0,23396,3424,5556,956.0,14362,859.0
3,2005,01001000,1,01,0101,1,0,66970,58379,8591,...,49002,8218,767.0,48235.0,21000,4196,5028,1035.0,13910,3066.0
4,2009,01001000,1,01,0101,1,0,69075,60983,8092,...,46583,7238,1152.0,45431.0,12739,6530,7323,2334.0,11238,5267.0


Concatenate first and second vote datasets

In [219]:
dfs = [btw_es_sum, btw_zs_sum]
btw_eszs = pd.concat(dfs, sort=False)
btw_eszs = btw_eszs.sort_values(['AGS', 'year'])
btw_eszs.head(12)

,year,AGS,first_vote,Land,Kreis,Wahlkreis,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,FDP,GRÜNE,Andere,Union,Linke,second_vote
0,1994,01001000,1.0,01,0101,1,0,68987,61031,7956,...,7535,552.0,51827.0,25927,1013,3509,996.0,20382,0.0,NaN
0,1994,01001000,NaN,01,0101,1,0,68987,61031,7956,...,7535,556.0,51823.0,24172,3404,4651,1059.0,17837,700.0,1.0
1,1998,01001000,1.0,01,0101,1,0,65755,58545,7210,...,6678,673.0,49887.0,27639,861,2000,897.0,17854,636.0,NaN
1,1998,01001000,NaN,01,0101,1,0,65755,58545,7210,...,6678,698.0,49862.0,25490,3182,3815,1925.0,14538,912.0,1.0
2,2002,01001000,1.0,01,0101,1,0,65740,57788,7949,...,7573,503.0,48551.0,26309,2038,2833,194.0,16418,759.0,NaN
2,2002,01001000,NaN,01,0101,1,0,65740,57788,7949,...,7573,501.0,48553.0,23396,3424,5556,956.0,14362,859.0,1.0
3,2005,01001000,1.0,01,0101,1,0,66970,58379,8591,...,8218,758.0,48244.0,24261,1601,2729,387.0,16954,2312.0,NaN
3,2005,01001000,NaN,01,0101,1,0,66970,58379,8591,...,8218,767.0,48235.0,21000,4196,5028,1035.0,13910,3066.0,1.0
4,2009,01001000,1.0,01,0101,1,0,69075,60983,8092,...,7238,1204.0,45379.0,16552,4253,5520,498.0,13859,4697.0,NaN
4,2009,01001000,NaN,01,0101,1,0,69075,60983,8092,...,7238,1152.0,45431.0,12739,6530,7323,2334.0,11238,5267.0,1.0


Add indicator for vote type (polling office, postal, other)

In [220]:
#btw_eszs['vote_type'] = np.where(btw_eszs['Bezirksart'] > 5 , 'other', np.nan) # 6 = Sonderwahlbezirk, 8 = Bezirke für Wahlberechtigte ohne nähere Angabe
#btw_eszs['vote_type'] = np.where(btw_eszs['Bezirksart'] == 5, 'postal', btw_eszs['vote_type'])
#btw_eszs['vote_type'] = np.where(btw_eszs['Bezirksart'] == 0, 'office', btw_eszs['vote_type'])

View balacing of panel

In [221]:
btw_eszs.year.value_counts()

1994    22334
2009    22326
1998    22322
2005    22320
2002    22318
2013    22318
2017    22314
Name: year, dtype: int64

Replace Land values with corresponding state abbreviation

In [222]:
btw_eszs['Land'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16'], dtype=object)

In [223]:
states = {'01': 'SH', '02': 'HH', '03': 'NI', '04': 'HB', '05': 'NW', '06': 'HE', '07': 'RP', '08': 'BW', \
         '09': 'BY', '10': 'SL', '11': 'BE', '12': 'BB', '13': 'MV', '14': 'SN', '15': 'ST', '16': 'TH'}
btw_eszs['state_id'] = btw_eszs['Land']
for key, value in states.items():
    btw_eszs['Land'] = np.where(btw_eszs['Land'] == key, value, btw_eszs['Land'])
btw_eszs['Land'].unique()

array(['SH', 'HH', 'NI', 'HB', 'NW', 'HE', 'RP', 'BW', 'BY', 'SL', 'BE',
       'BB', 'MV', 'SN', 'ST', 'TH'], dtype=object)

Drop not needed columns and rename

In [224]:
btw_eszs.rename(columns = {'Wahlberechtigte (A)': 'Wahlberechtigte', 'GRÜNE': 'Grüne', 'Wähler (B)': 'Wähler'}, inplace = True)
columns_to_drop = ['Bezirksart',
       'Wahlberechtigte ohne Sperrvermerk (A1)',
       'Wahlberechtigte mit Sperrvermerk (A2)',
       'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)',
       'Wähler mit Wahlschein (B1)']
btw_eszs.drop(columns_to_drop, axis=1, inplace=True)
btw_eszs.shape

(156252, 18)

Since vote percentages are calculated based on the 'valid' votes and there are some rows with valid == 0, calculate the vote shares for these (Voters - Invalid)

In [225]:
btw_eszs.loc[:, 'Gültig'] = np.where(btw_eszs['Gültig'] == 0, btw_eszs['Wähler'] - btw_eszs['Ungültig'], btw_eszs['Gültig'])

Check summation of party and invalid votes

In [226]:
parties = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Ungültig']
btw_eszs.loc[:, 'votes'] = btw_eszs[parties].sum(axis=1)
btw_eszs.loc[:, 'votes'] = btw_eszs['votes'].astype(int) #convert to integer
print(btw_eszs['votes'].equals(btw_eszs['Wähler'])) # check if votes == Wähler
btw_eszs.drop(['votes'], axis=1, inplace=True)

True


Check summation of valid and invalid votes

In [227]:
votes = ['Gültig', 'Ungültig']
btw_eszs['votes'] = btw_eszs[votes].sum(axis=1)
btw_eszs['votes'] = btw_eszs['votes'].astype(int) #convert to integer
print(btw_eszs['votes'].equals(btw_eszs['Wähler'])) # check if votes == Wähler
btw_eszs.drop(['votes', 'Ungültig'], axis=1, inplace=True)

True


Calculate vote shares in percent

In [228]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    btw_eszs.loc[:, party] = (btw_eszs[party] / btw_eszs['Gültig'])*100
btw_eszs.loc[:, 'Wähler'] = (btw_eszs['Wähler'] / btw_eszs['Wahlberechtigte'])*100
btw_eszs.head(10)

,year,AGS,first_vote,Land,Kreis,Wahlkreis,Wahlberechtigte,Wähler,Gültig,SPD,FDP,Grüne,Andere,Union,Linke,second_vote,state_id
0,1994,01001000,1.0,SH,0101,1,68987,75.925899,51827.0,50.026048,1.954580,6.770602,1.921778,39.326992,0.000000,NaN,01
0,1994,01001000,NaN,SH,0101,1,68987,75.925899,51823.0,46.643382,6.568512,8.974780,2.043494,34.419080,1.350752,1.0,01
1,1998,01001000,1.0,SH,0101,1,65755,76.891491,49887.0,55.403211,1.725901,4.009060,1.798064,35.788883,1.274881,NaN,01
1,1998,01001000,NaN,SH,0101,1,65755,76.891491,49862.0,51.121094,6.381613,7.651117,3.860655,29.156472,1.829048,1.0,01
2,2002,01001000,1.0,SH,0101,1,65740,74.618193,48551.0,54.188379,4.197648,5.835101,0.399580,33.815987,1.563305,NaN,01
2,2002,01001000,NaN,SH,0101,1,65740,74.618193,48553.0,48.186518,7.052087,11.443165,1.968982,29.580047,1.769201,1.0,01
3,2005,01001000,1.0,SH,0101,1,66970,73.170076,48244.0,50.288119,3.318547,5.656662,0.802172,35.142194,4.792306,NaN,01
3,2005,01001000,NaN,SH,0101,1,66970,73.170076,48235.0,43.536851,8.699077,10.423966,2.145745,28.837981,6.356380,1.0,01
4,2009,01001000,1.0,SH,0101,1,69075,67.438292,45379.0,36.475021,9.372177,12.164217,1.097424,30.540558,10.350603,NaN,01
4,2009,01001000,NaN,SH,0101,1,69075,67.438292,45431.0,28.040325,14.373445,16.118950,5.137461,24.736413,11.593405,1.0,01


Clean dataset and export

In [229]:
btw_eszs = btw_eszs[['AGS', 'year', 'Land', 'state_id', 'Kreis', 'Wahlkreis', 'first_vote', 'second_vote', 'Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw_eszs = btw_eszs.set_index(['AGS', 'year'])
btw_eszs.to_csv(f'{path}/data/btw_eszs.csv', encoding = 'utf-8-sig')

Group Erst- and Zweitstimme

In [230]:
btw = btw_eszs.reset_index()
cols_to_keep = ['Land', 'state_id', 'Kreis', 'Wahlkreis',]
cols_to_sum = ['Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
btw_grouped = btw.groupby(['AGS', 'year'], as_index=False).agg({**{e:'first' for e in cols_to_keep}, **{e:'mean' for e in cols_to_sum}})
btw = btw_grouped.copy()

Drop obs. with missing values for election results

In [231]:
btw = btw.dropna(axis='rows', how='any', subset=['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'])
btw.shape

(78123, 13)

Merge with treatment dataset

In [232]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})
btw_treat = pd.merge(btw, treatment, on='AGS', how='outer', indicator=True)
btw_treat['_merge'].value_counts()

both          78110
right_only      219
left_only        13
Name: _merge, dtype: int64

Clean data and export

In [233]:
btw_treat = btw_treat[btw_treat['_merge'] == 'both']
btw_treat = btw_treat.drop(columns=['_merge'])
btw_treat = btw_treat.set_index(['AGS', 'GEN', 'year'])
btw_treat.to_csv(f'{path}/data/btw_treat.csv', encoding = 'utf-8-sig')

Merge in control variables

In [234]:
btw_treat = btw_treat.reset_index()
controls = pd.read_csv(f'{path}/data/controls.csv', converters = {'AGS': str})
btw_control = pd.merge(btw_treat, controls, on=['AGS', 'year'], how='left', indicator=True)
btw_control['_merge'].value_counts()

both          78110
left_only         0
right_only        0
Name: _merge, dtype: int64

In [235]:
btw_control = btw_control.set_index(['AGS', 'year'])
btw_control = btw_control.drop(columns=['_merge'])

Add dummy for East Germany and South

In [236]:
btw_control['Land'].unique()

array(['SH', 'HH', 'NI', 'HB', 'NW', 'HE', 'RP', 'BW', 'BY', 'SL', 'BE',
       'BB', 'MV', 'SN', 'ST', 'TH'], dtype=object)

In [237]:
east = ['BE', 'BB', 'MV', 'SN', 'ST', 'TH']
south = ['BW', 'BY']
btw_control['east'] = np.where(btw_control['Land'].isin(east), 1, 0)
btw_control['south'] = np.where(btw_control['Land'].isin(south), 1, 0)

In [238]:
btw_control['east'].value_counts()

0    59147
1    18963
Name: east, dtype: int64

In [239]:
btw_control['south'].value_counts()

0    56006
1    22104
Name: south, dtype: int64

Export dataset

In [240]:
btw_control.to_csv(f'{path}/data/btw_control.csv', encoding = 'utf-8-sig')
btw_control.head()

GEN Land state_id Kreis  Wahlkreis     Wähler  \
AGS      year                                                          
01001000 1994.0  Flensburg   SH       01  0101        1.0  75.925899   
         1998.0  Flensburg   SH       01  0101        1.0  76.891491   
         2002.0  Flensburg   SH       01  0101        1.0  74.618193   
         2005.0  Flensburg   SH       01  0101        1.0  73.170076   
         2009.0  Flensburg   SH       01  0101        1.0  67.438292   

                     Union        SPD        FDP      Linke  ...  \
AGS      year                                                ...   
01001000 1994.0  36.873036  48.334715   4.261546   0.675376  ...   
         1998.0  32.472677  53.262153   4.053757   1.551965  ...   
         2002.0  31.698017  51.187449   5.624868   1.666253  ...   
         2005.0  31.990087  46.912485   6.008812   5.574343  ...   
         2009.0  27.638486  32.257673  11.872811  10.972004  ...   

                 never_treated  pop_density unemployed  foreign  catholic  \
AGS      year                                                               
01001000 1994.0            0.0  1558.114812        NaN      NaN       NaN   
         1998.0            0.0  1501.417434   6.825423      NaN       NaN   
         2002.0            0.0  1502.305782   5.994994      NaN       NaN   
         2005.0            0.0  1526.782547   8.737221      NaN       NaN   
         2009.0            0.0  1559.746211   6.236017  6.23526  6.345887   

                 avg_age  female  income_pc  east south  
AGS      year                                            
01001000 1994.0      NaN     NaN        NaN     0     0  
         1998.0      NaN   51.73        NaN     0     0  
         2002.0    41.13   51.60        NaN     0     0  
         2005.0    41.25   51.25        NaN     0     0  
         2009.0    41.52   50.68  10.204979     0     0  

[5 rows x 30 columns]